In [92]:
import praw
import pandas as pd
import datetime as dt

import sqlite3
import sqlalchemy as db
from sqlalchemy import create_engine

### Defining major functions for scrapping Reddit

In [104]:
def get_reddit(client_id,client_secret,user_agent):
    reddit = praw.Reddit(client_id=client_id,client_secret = client_secret,user_agent=user_agent)
    return reddit

In [106]:
def get_posts_result(reddit,keyword,len_of_result):
    posts = []
    ml_subreddit = reddit.subreddit(keyword)
    for post in ml_subreddit.new():
        if not (post.is_self & len(posts)==len_of_result):
            posts.append([post.id, post.subreddit, post.title, post.selftext, post.url, dt.datetime.now(),post.author,post.created_utc,post.comments])
    posts_df=pd.DataFrame(posts,columns=['post_id','subreddit_name','post_title','post_body','visual_content_url','scrap_time','post_author','post_publish_date','post_comments'])
    return posts_df

In [109]:
def transform_posts_results(posts_df):
    posts_df['post_publish_date'] = pd.to_datetime(posts_df['post_publish_date'],unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    col_name= ["post_id","subreddit_name",'post_title','post_body','visual_content_url','post_author','post_comments']
    posts_df[col_name] = posts_df[col_name].astype(str)
    return posts_df

### Execution

In [102]:
# Setting up the reddit authentication information
client_id="rKCfOb1SzwpPQQ"
client_secret = 'Ktrwjri1ugder7wWFRuDwPjitll5Zg'
user_agent='FixFake_Scraper'

In [105]:
reddit = get_reddit(client_id,client_secret,user_agent)

In [107]:
# Getting the scrap result dataframe
posts_df = get_posts_result(reddit,'Shoes',100)

In [110]:
# Transforming the dataframe
posts_df = transform_posts_results(posts_df)

In [112]:
# Exporting dataframe to database
engine = create_engine('sqlite:///reddit_shoes.db',echo=True)
sqlite_connection = engine.connect()
sqlite_table="reddit_shoes"
posts_df.to_sql(sqlite_table,sqlite_connection,if_exists='replace')

2020-12-19 21:57:59,131 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-19 21:57:59,133 INFO sqlalchemy.engine.base.Engine ()
2020-12-19 21:57:59,138 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-19 21:57:59,140 INFO sqlalchemy.engine.base.Engine ()


In [115]:
# View the database info 
table_df = pd.read_sql_table("reddit_shoes",con=engine)
print(table_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
index                 100 non-null int64
post_id               100 non-null object
subreddit_name        100 non-null object
post_title            100 non-null object
post_body             100 non-null object
visual_content_url    100 non-null object
scrap_time            100 non-null datetime64[ns]
post_author           100 non-null object
post_publish_date     100 non-null object
post_comments         100 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 7.9+ KB
None


### Below is the profile results

In [91]:
import cProfile

In [117]:
cProfile.run("reddit = get_reddit(client_id,client_secret,user_agent)")

         228 function calls in 0.044 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.039    0.039 <ipython-input-104-ff6253f4c502>:1(get_reddit)
        1    0.000    0.000    0.039    0.039 <string>:1(<module>)
        2    0.001    0.001    0.001    0.001 _collections.py:44(__init__)
        6    0.000    0.000    0.000    0.000 _collections_abc.py:657(get)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:824(update)
        1    0.000    0.000    0.000    0.000 abc.py:137(__instancecheck__)
        2    0.000    0.000    0.003    0.001 adapters.py:113(__init__)
        2    0.000    0.000    0.002    0.001 adapters.py:146(init_poolmanager)
        2    0.000    0.000    0.000    0.000 adapters.py:58(__init__)
        1    0.000    0.000    0.000    0.000 auth.py:102(__init__)
        1    0.000    0.000    0.000    0.000 auth.py:12(__init__)
        1    0.000    0.000   

In [116]:
!pip freeze > requirements.txt